# Exercise01 : Prepare Config Settings

Set your config in your current project folder.    
(Before starting, you must create your Azure Machine Learning workspace in Azure Portal. See [readme](https://github.com/tsmatz/azureml-tutorial/) for set up.)

*back to [index](https://github.com/tsmatz/azureml-tutorial/)*

## Initialize config settings (Connect to AML workspace)

You must create config setting into your project directory.

Copy your Azure Machine Learning workspace name, subscription ID, and resource group name in Azure Machine Learning overview page on Azure Portal, and fill the following parameters.    
If you're asked to login Azure with device login UI (https://microsoft.com/devicelogin), please open your browser and proceed to login.

This is needed **only once** in your working directory. (The result was written in ```.azureml/config.json``` and used same configuration in this directory.)

In [1]:
from azureml.core import Workspace
ws = Workspace(
  workspace_name = "{AML WORKSPACE NAME}",
  subscription_id = "{SUBSCRIPTION ID}",
  resource_group = "{RESOURCE GROUP NAME}")
ws.write_config()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ASC8W7F83 to authenticate.
Interactive authentication successfully completed.


Check whether you have connected to AML workspace. (You can see the connected config settings.)

In [ ]:
ws = Workspace.from_config()
ws.get_details()

## Switch from current connection to another (Optional)

When you want to switch to AML workspace in another tenant, run the following command.

In [ ]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
interactive_auth = InteractiveLoginAuthentication(
    tenant_id="{TENANT-ID}")
ws = Workspace(
    workspace_name = "{AML WORKSPACE NAME}",
    subscription_id = "{SUBSCRIPTION ID}",
    resource_group = "{RESOURCE GROUP NAME}",
    auth=interactive_auth)

## Connect to AML workspace with silent login (Optional)

For integrating with the automated operations (MLOps), you can use a service principal to connect ML workspace without login UI.

For preparation, please add a service principal with contributor rights to AML resource group, and get a secret for this service principal.<br>
(**You need to install Azure CLI to run this command.** You can also do this task in [Azure Portal UI](Https://portal.azure.com/).)

In [ ]:
!az login

In [ ]:
# Replace {SERVICE PRINCIPAL NAME}, {SUBSCRIPTION ID} and {RESOURCE GROUP NAME}.
# {SERVICE PRINCIPAL NAME} is an arbitrary string.
!az ad sp create-for-rbac --name {SERVICE PRINCIPAL NAME} \
    --role contributor \
    --scopes /subscriptions/{SUBSCRIPTION ID}/resourceGroups/{RESOURCE GROUP NAME}

This will generate the following JSON output.<br>
Copy {CLIENT ID}, {CLIENT SECRET}, and {TENANT ID}.

```
{
  "appId": "{CLIENT ID}",
  "displayName": "{SERVICE PRINCIPAL NAME}",
  "name": "{GUID}",
  "password": "{CLIENT SECRET}",
  "tenant": "{TENANT ID}"
}
```

By running the following command, now you can get AML workspace connection using a service principal credentials.<br>
Please specify "AzureCloud", "AzureUSGovernment", or "AzureChinaCloud" in ```cloud``` property.

In [ ]:
from azureml.core.authentication import ServicePrincipalAuthentication

sp_auth = ServicePrincipalAuthentication(
    tenant_id = "{TENANT ID}",
    service_principal_id = "{CLIENT ID}",
    service_principal_password = "{CLIENT SECRET}",
    cloud = "AzureCloud"
)
ws = Workspace.get(
    name = "{AML WORKSPACE NAME}",
    subscription_id = "{SUBSCRIPTION ID}",
    resource_group = "{RESOURCE GROUP NAME}",
    auth = sp_auth
)